<h1>Antoine Equation</h1>

In [1]:
# ----- Fitting the Curve to get the Antoine Equation -----

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# fitting logP vs T
P_data = np.array([1, 10, 100, 1000, 10000, 100000])  # Pressure in Pa
T_data = np.array([37, 41, 46, 53, 62, 77])  # Temperature in K

# Antoine equation: log10(P) = A - B/(T + C)
def antoine_original(T, A, B, C):
    return A - B/(T + C)

# Fitting equation
logP_data = np.log10(P_data)
p0 = [5.0, 300.0, -10.0]  # Initial guess
popt, pcov = curve_fit(antoine_original, T_data, logP_data, p0=p0)

A, B, C = popt
print(f"Antoine parameters (original form):")
print(f"A = {A:.4f}")
print(f"B = {B:.4f}")
print(f"C = {C:.4f}")

# Function to calculate temperature from pressure
def P_to_T(P, A, B, C):
    return B/(A - np.log10(P)) - C

# Calculate boiling point
P_ambient = 102100  # Pa
T_boiling = P_to_T(P_ambient, A, B, C)
print(f"Boiling point at {P_ambient} Pa: {T_boiling:.2f} K")

# Create comprehensive results table
print(f"\n{'Pressure (Pa)':<12} {'Pressure (mbar)':<15} {'Temperature (K)':<15}")
print("-" * 45)
for P in [1, 10, 100, 1000, 10000, 100000]:
    T_calc = P_to_T(P, A, B, C)
    print(f"{P:<12} {P/100:<15.1f} {T_calc:<15.2f}")

# Show ambient pressure result
print(f"{P_ambient:<12} {P_ambient/100:<15.1f} {T_boiling:<15.2f}")

Antoine parameters (original form):
A = 8.9287
B = 278.8647
C = -5.7876
Boiling point at 102100 Pa: 76.93 K

Pressure (Pa) Pressure (mbar) Temperature (K)
---------------------------------------------
1            0.0             37.02          
10           0.1             40.96          
100          1.0             46.04          
1000         10.0            52.82          
10000        100.0           62.37          
100000       1000.0          76.77          
102100       1021.0          76.93          


<h1>Temperature-Voltage Relationship</h1>

In [2]:
import numpy as np
from scipy import stats

def voltage_to_temp_type_t(voltage_mv):
    """
    Convert voltage to temperature for Type T thermocouple
    using NIST ITS-90 polynomial coefficients for range -100°C to 0°C
    """
    # Coefficients for Type T thermocouple for range -100°C to 0°C
    coefficients = [
        0.0000000E+00,
        2.5949192E+01, 
        -2.1316967E-01,
        7.9018692E-01,
        4.2527777E-01,
        1.3304473E-01,
        2.0241446E-02,
        1.2668171E-03
    ]
    
    # Calculate temperature using polynomial
    temp = 0.0
    for i, coef in enumerate(coefficients):
        temp += coef * (voltage_mv ** i)
    
    return temp

def main():
    # Measured thermocouple voltages from the experiment
    v_tc_measurements = [
        -4.98, -4.419, -3.95, -3.49, -3.08, -2.5, -2.07,
        -1.43, -1.07, -0.49, -0.02, 0.53, 1.01
    ]
    
    # Reference junction parameters
    v_ref_mv = -0.014  # 0°C corresponds to -0.014mV for Type T
    t_ref_c = 0.0      # Reference junction temperature
    
    results = []
    
    print(f"{'V_TC (mV)':<10} {'V_corrected (mV)':<15} {'Temperature (°C)':<18} {'Temperature (K)':<15}")
    print("-" * 60)
    
    for v_tc in v_tc_measurements:
        # Apply cold junction compensation
        v_corrected = v_tc + v_ref_mv
        
        # Convert corrected voltage to temperature in Celsius
        temperature_c = voltage_to_temp_type_t(v_corrected)
        temperature_k = temperature_c + 273.15  # Convert to Kelvin
        
        results.append({
            'V_TC_mV': v_tc,
            'V_corrected_mV': v_corrected,
            'Temperature_C': temperature_c,
            'Temperature_K': temperature_k
        })
        
        print(f"{v_tc:<10.3f} {v_corrected:<15.3f} {temperature_c:<18.2f} {temperature_k:<15.2f}")
    
    return results

if __name__ == "__main__":
    print("Type T Thermocouple Analysis")
    print("Reference: 0°C = -0.014mV for Type T thermocouple")
    print()
    
    results = main()


Type T Thermocouple Analysis
Reference: 0°C = -0.014mV for Type T thermocouple

V_TC (mV)  V_corrected (mV) Temperature (°C)   Temperature (K)
------------------------------------------------------------
-4.980     -4.994          -166.22            106.93         
-4.419     -4.433          -140.60            132.55         
-3.950     -3.964          -121.59            151.56         
-3.490     -3.504          -104.46            168.69         
-3.080     -3.094          -90.16             182.99         
-2.500     -2.514          -71.21             201.94         
-2.070     -2.084          -57.92             215.23         
-1.430     -1.444          -39.11             234.04         
-1.070     -1.084          -28.97             244.18         
-0.490     -0.504          -13.21             259.94         
-0.020     -0.034          -0.88              272.27         
0.530      0.516           13.48              286.63         
1.010      0.996           26.98              300.13